# Changing the Leetcode stats in my CV

## 1. get data from Leetcode

In [97]:
import requests
import json
from bs4 import BeautifulSoup

In [98]:

def get_leetcode_profile_data(username):
    url = f"https://leetcode-stats-api.herokuapp.com/{username}/"
    response = requests.get(url)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Decode the byte content to a string
        response_content = response.content.decode('utf-8')

        # Parse the JSON string into a dictionary
        profile_data = json.loads(response_content)

        return profile_data
    else:
        print("Failed to retrieve data.")
        return None


In [99]:
username = 'josefpolasek'
profile_data = get_leetcode_profile_data(username)
# print(profile_data)

total = profile_data["totalSolved"]
easy = profile_data["easySolved"]
medium = profile_data["mediumSolved"]
hard = profile_data["hardSolved"]
rank = profile_data["ranking"]

print(f"Total Problems Solved: {total}")
print(f"Easy: {easy}")
print(f"Medium: {medium}")
print(f"Hard: {hard}")
print(f"Rank: {rank:,}")

Total Problems Solved: 34
Easy: 29
Medium: 5
Hard: 0
Rank: 1,845,022


## 2. Update the CV

In [100]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

SERVICE_ACCOUNT_FILE = 'credentials.json'

# Create credentials object
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize the Google Drive service
drive_service = build('drive', 'v3', credentials=credentials)

# Initialize the Google Docs service
docs_service = build('docs', 'v1', credentials=credentials)


### Copy the document

In [101]:
document_id = '1bCtSLi71qrT03579jhfNyS6SOvdSKh7RUtPJMBMAGtY'

copy_title = '5'
body = {
    'name': copy_title
}
drive_response = drive_service.files().copy(
    fileId=document_id, body=body).execute()
document_copy_id = drive_response.get('id')


### Replace placeholders

In [102]:
replacements = {
    '{easy_solved}': str(easy),
    '{medium_solved}': str(medium),
    '{hard_solved}': str(hard)
}

# Create a list of requests to replace each placeholder
requests = []
for placeholder, value in replacements.items():
    requests.append({
        'replaceAllText': {
            'containsText': {
                'text': placeholder,
                'matchCase': True
            },
            'replaceText': value,
        }
    })

result = docs_service.documents().batchUpdate(
    documentId=document_copy_id, body={'requests': requests}).execute()


### Delete duplicate CV.pdf

In [103]:
folder_id = '1DxFHYlVDQNhuVsLMYT2nJJFgwNzAYcfq'
file_name = 'CV.pdf'

query = f'"{folder_id}" in parents and trashed=false and name="{file_name}"'
results = drive_service.files().list(q=query).execute()
files = results.get('files', [])
if not files:
    print("No file found.")
else:
    file_id = files[0]['id']
    drive_service.files().delete(fileId=file_id).execute()

### Save as PDF

In [104]:
pdf_file = drive_service.files().export(fileId=document_copy_id, mimeType='application/pdf').execute()

### Upload the PDF

In [105]:
from googleapiclient.http import MediaIoBaseUpload
import io

# upload it
file_metadata = {
    'name': 'CV.pdf',
    'parents': ['1DxFHYlVDQNhuVsLMYT2nJJFgwNzAYcfq']
}

pdf_file_io = io.BytesIO(pdf_file)
media = MediaIoBaseUpload(pdf_file_io, mimetype='application/pdf')

uploaded_file = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()



### Delete the copy

In [106]:
drive_service.files().delete(fileId=document_copy_id).execute()

''